# Qiskit Nature HDF5 Integration

In [1]:
from qiskit_nature.hdf5 import load_from_hdf5, save_to_hdf5
from qiskit_nature.drivers.second_quantization import PySCFDriver, GaussianForcesDriver
from qiskit_nature.properties.second_quantization.vibrational.bases import HarmonicBasis

## Electronic Structure

Let's start with a simple electronic structure example:

In [2]:
electronic_driver = PySCFDriver(
    atom="H 0.0 0.0 0.0; Li 0.0 0.0 1.3",
    basis="sto-3g",
)

electronic_structure_driver_result = electronic_driver.run()

In [3]:
print(electronic_structure_driver_result)

ElectronicStructureDriverResult:
	DriverMetadata:
		Program: PYSCF
		Version: 2.0.1
		Config:
			atom=H 0.0 0.0 0.0; Li 0.0 0.0 1.3
			unit=Angstrom
			charge=0
			spin=0
			basis=sto-3g
			method=rhf
			conv_tol=1e-09
			max_cycle=50
			init_guess=minao
			max_memory=4000
			
	ElectronicBasisTransform:
		Initial basis: atomic
		Final basis: molecular
		Alpha coefficients:
		[0, 0] = 0.015338518759435149
		[0, 1] = 0.5397949633009688
		[0, 2] = -0.1274188208260597
		[0, 5] = 1.3433648810566434
		[1, 0] = 0.9903633517919679
		[1, 1] = -0.19621595483094248
		[1, 2] = -0.20485100732030045
		[1, 5] = 0.028152840369250906
		[2, 0] = 0.02918799315633499
		[2, 1] = 0.4249977435147273
		[2, 2] = 0.8090567055477585
		[2, 5] = -0.8277204736828909
		[3, 4] = 1.0000000000000002
		[4, 3] = 1.0000000000000002
		[5, 0] = 0.009887477240996655
		[5, 1] = -0.35711825878454834
		[5, 2] = 0.6094847431614152
		[5, 5] = 1.0712492005242007
	ParticleNumber:
		12 SOs
		2 alpha electrons
			orbital occupation: 

We can save this in an HDF5 file like so:

In [4]:
save_to_hdf5(electronic_structure_driver_result, "tmp_electronic_structure_driver_result.hdf5", replace=True)

And can then later reconstruct it like so:

In [5]:
new_electronic_structure_driver_result = load_from_hdf5("tmp_electronic_structure_driver_result.hdf5")

In [6]:
print(new_electronic_structure_driver_result)

ElectronicStructureDriverResult:
	AngularMomentum:
		12 SOs
	DriverMetadata:
		Program: PYSCF
		Version: 2.0.1
		Config:
			atom=H 0.0 0.0 0.0; Li 0.0 0.0 1.3
			unit=Angstrom
			charge=0
			spin=0
			basis=sto-3g
			method=rhf
			conv_tol=1e-09
			max_cycle=50
			init_guess=minao
			max_memory=4000
			
	ElectronicBasisTransform:
		Initial basis: atomic
		Final basis: molecular
		Alpha coefficients:
		[0, 0] = 0.015338518759435149
		[0, 1] = 0.5397949633009688
		[0, 2] = -0.1274188208260597
		[0, 5] = 1.3433648810566434
		[1, 0] = 0.9903633517919679
		[1, 1] = -0.19621595483094248
		[1, 2] = -0.20485100732030045
		[1, 5] = 0.028152840369250906
		[2, 0] = 0.02918799315633499
		[2, 1] = 0.4249977435147273
		[2, 2] = 0.8090567055477585
		[2, 5] = -0.8277204736828909
		[3, 4] = 1.0000000000000002
		[4, 3] = 1.0000000000000002
		[5, 0] = 0.009887477240996655
		[5, 1] = -0.35711825878454834
		[5, 2] = 0.6094847431614152
		[5, 5] = 1.0712492005242007
	ElectronicDipoleMoment:
		DipoleMomentX
	

## Intermission

Why do we need this?
- we can run a piece of our simulation on another computer (HPC) and transfer the HDF5
- we can run an initial simulation of a large system, store the result, and use the same starting point for multiple simulations of e.g. different subsystems (think `ActiveSpaceTransformer`)

## Vibrational Structure

In the past, this was only possible with `QMolecule` objects but now this also works seemlessly for vibrational structure cases:

In [7]:
vibrational_driver = GaussianForcesDriver(
    logfile="test/drivers/second_quantization/gaussiand/test_driver_gaussian_log_C01.txt"
)

vibrational_structure_driver_result = vibrational_driver.run()
vibrational_structure_driver_result.basis = HarmonicBasis(num_modals_per_mode=2)

In [8]:
print(vibrational_structure_driver_result)

VibrationalStructureDriverResult:
	HarmonicBasis:
		Modals: 2:
	VibrationalEnergy:
		HarmonicBasis:
			Modals: 2
		1-Body Terms:
			<sparse integral list with 13 entries>
			(2, 2) = 352.3005875
			(-2, -2) = -352.3005875
			(1, 1) = 631.6153975
			(-1, -1) = -631.6153975
			(4, 4) = 115.653915
			... skipping 8 entries
		2-Body Terms:
			<sparse integral list with 11 entries>
			(1, 1, 2) = -88.2017421687633
			(4, 4, 2) = 42.675273102831454
			(3, 3, 2) = 42.675273102831454
			(1, 1, 2, 2) = 4.9425425
			(4, 4, 2, 2) = -4.194299375
			... skipping 6 entries
		3-Body Terms:
			<sparse integral list with 0 entries>
	OccupiedModals:
		HarmonicBasis:
			Modals: 2


In [9]:
save_to_hdf5(vibrational_structure_driver_result, "tmp_vibrational_structure_driver_result.hdf5", replace=True)

In [10]:
new_vibrational_structure_driver_result = load_from_hdf5("tmp_vibrational_structure_driver_result.hdf5")

In [11]:
print(new_vibrational_structure_driver_result)

VibrationalStructureDriverResult:
	HarmonicBasis:
		Modals: 2:
	OccupiedModals:
		HarmonicBasis:
			Modals: 2
	VibrationalEnergy:
		HarmonicBasis:
			Modals: 2
		1-Body Terms:
			<sparse integral list with 13 entries>
			(2, 2) = 352.3005875
			(-2, -2) = -352.3005875
			(1, 1) = 631.6153975
			(-1, -1) = -631.6153975
			(4, 4) = 115.653915
			... skipping 8 entries
		2-Body Terms:
			<sparse integral list with 11 entries>
			(1, 1, 2) = -88.2017421687633
			(4, 4, 2) = 42.675273102831454
			(3, 3, 2) = 42.675273102831454
			(1, 1, 2, 2) = 4.9425425
			(4, 4, 2, 2) = -4.194299375
			... skipping 6 entries
		3-Body Terms:
			<sparse integral list with 0 entries>


## General Design

In [12]:
from __future__ import annotations
from typing import Protocol

import h5py


class HDF5Storable(Protocol):
    """A Protocol implemented by those classes which support conversion methods for HDF5."""

    def to_hdf5(self, parent: h5py.Group) -> None:
        """Stores this instance in an HDF5 group inside of the provided parent group.
        ...
        """
        ...

    @staticmethod
    def from_hdf5(h5py_group: h5py.Group) -> HDF5Storable:
        """Constructs a new instance from the data stored in the provided HDF5 group.
        ...
        """
        ...

In [13]:
class Molecule:
    VERSION = 1
    
    def to_hdf5(self, parent: h5py.Group) -> None:
        group = parent.require_group(self.__class__.__name__)
        group.attrs["__class__"] = self.__class__.__name__
        group.attrs["__module__"] = self.__class__.__module__
        group.attrs["__version__"] = self.VERSION

        geometry_group = group.create_group("geometry", track_order=True)
        for idx, geom in enumerate(self._geometry):
            symbol, coords = geom
            geometry_group.create_dataset(str(idx), data=coords)
            geometry_group[str(idx)].attrs["symbol"] = symbol

        group.attrs["units"] = self.units.value
        group.attrs["multiplicity"] = self.multiplicity
        group.attrs["charge"] = self.charge

        if self._masses:
            group.create_dataset("masses", data=self._masses)

    @staticmethod
    def from_hdf5(h5py_group: h5py.Group) -> Molecule:
        geometry = []
        for atom in h5py_group["geometry"].values():
            geometry.append((atom.attrs["symbol"], list(atom[...])))

        units: UnitsType
        for unit in UnitsType:
            if unit.value == h5py_group.attrs["units"]:
                units = unit
                break
        else:
            units = UnitsType.ANGSTROM

        multiplicity = h5py_group.attrs["multiplicity"]
        charge = h5py_group.attrs["charge"]

        masses = None
        if "masses" in h5py_group.keys():
            masses = list(h5py_group["masses"])

        return Molecule(
            geometry,
            multiplicity=multiplicity,
            charge=charge,
            units=units,
            masses=masses,
        )

In [14]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright